# Quick Start for PanCollection

###  Step1. Install package `pancollection`

In [4]:
!pip install pancollection --upgrade

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


In [ ]:
!pip show udl_vis
!pip show pancollection

### Step2. Detect GPU

In [5]:
!nvidia-smi

Fri Jun  9 18:59:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.36       Driver Version: 512.36       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   61C    P8     6W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Step3. Test codes in pretrained chekpoint

In [ ]:
import argparse
from pancollection.configs.configs import TaskDispatcher
import os

class parser_args(TaskDispatcher, name='FusionNet'):
    def __init__(self, cfg=None, **kwargs):
        super(parser_args, self).__init__()

        if cfg is None:
            from pancollection.configs.configs import panshaprening_cfg
            cfg = panshaprening_cfg()

        
        script_path = os.path.dirname(os.getcwd())
        root_dir = script_path.split(cfg.task)[0]

        model_path = f'.pth.tar'
        dataset_name = kwargs["dataset_name"] if 'dataset_name' in kwargs else "wv3"
        use_resume = kwargs["use_resume"] if 'use_resume' in kwargs else True

        parser = argparse.ArgumentParser(description='PyTorch Pansharpening Training')
        # * Logger
        parser.add_argument('--out_dir', metavar='DIR', default=f'{root_dir}/results/{cfg.task}',
                            help='path to save model')
        parser.add_argument('--mode', default=argparse.SUPPRESS, help='protective declare, please ignore it')

        parser.add_argument('--lr', default=2e-4, type=float)  # 1e-4 2e-4 8
        # parser.add_argument('--lr_scheduler', default=True, type=bool)
        parser.add_argument('--samples_per_gpu', default=32, type=int,  # 8
                            metavar='N', help='mini-batch size (default: 256)')
        parser.add_argument('--print-freq', '-p', default=50, type=int,
                            metavar='N', help='print frequency (default: 10)')
        parser.add_argument('--seed', default=1, type=int,
                            help='seed for initializing training. ')
        parser.add_argument('--epochs', default=400, type=int)
        parser.add_argument('--workers_per_gpu', default=0, type=int)
        parser.add_argument('--resume_from',
                            default=model_path,
                            type=str, metavar='PATH',
                            help='path to latest checkpoint (default: none)')
        ##
        parser.add_argument('--arch', '-a', metavar='ARCH', default='FusionNet', type=str,
                            choices=['PanNet', 'DiCNN', 'PNN', 'FusionNet'])
        parser.add_argument('--dataset', default={'train': 'wv3', 'test': 'test_wv3_multiExm1.h5'}, type=str, # 'valid': 'wv3' , 'eval': 'wv3_multiExm.h5'
                            choices=[None, 'wv2', 'wv3', 'wv4', 'qb', 'gf2', 'wv3_OrigScale_multiExm1.h5', 'test_wv3_multiExm1.h5'],
                            help="performing evalution for patch2entire")
        parser.add_argument('--eval', default=False, type=bool,
                            help="performing evaluation out of training process, which can avoid dead loop !!")


        args = parser.parse_args(args=[])
        args.start_epoch = args.best_epoch = 1
        args.experimental_desc = 'Test'
        cfg.merge_args2cfg(args)
        cfg.save_fmt = "mat"
        # cfg.workflow = [('train', 10), ('val', 1)]
        cfg.workflow = [('train', 1)]
        # cfg.config = f"{script_path}/configs/hook_configs.py"
        cfg.use_tfb = False
        cfg.img_range = 2047.0 if dataset_name != "gf2" else 1023.0
        cfg.dataloader_name = "PanCollection_dataloader"  # PanCollection_dataloader, oldPan_dataloader, DLPan_dataloader
        cfg.merge_args2cfg(args)
        
        cfg.merge_from_dict(kwargs)  # dict is merged partially
        cfg.dataset = kwargs['dataset'] if 'dataset' in kwargs else cfg.dataset
        print(cfg.pretty_text)


        self.merge_from_dict(cfg)


In [2]:
%reload_ext autoreload
# %autoreload 2
%aimport udl_vis
%aimport pancollection


import pancollection as pan
cfg = pan.TaskDispatcher.new(task='pansharpening', mode='entrypoint', arch='FusionNet', 
                             dataset_name="gf2", use_resume=True, eval=True,
                             dataset={'train': 'wv3', 'test': 'test_wv3_multiExm1.h5'})
print(pan.TaskDispatcher._task)
pan.trainer.main(cfg, pan.build_model, pan.getDataSession)


d:\python\gitsync\udl_package\pancollection\pancollection\configs\configs.py:33: UserWarning: Note: FusionNet, DiCNN, PNN don't have high-pass filter
  warnings.warn(warning)
- Set random seed to 1


111 entrypoint
111 None
222 pansharpening
111 None
333 FusionNet
accumulated_step = 1
amp = None
amp_opt_level = 'O0'
arch = 'FusionNet'
backend = 'nccl'
clip_max_norm = 0
crop_batch_size = 128
device = 'cuda'
dist_url = 'env://'
global_rank = 0
gpu_ids = [0]
launcher = 'none'
load_model_strict = True
local_rank = 0
log_dir = 'logs'
mode = None
model_style = None
once_epoch = False
reg = True
reset_lr = False
resume_mode = 'best'
rgb_range = 255
save_top_k = 5
seed = 1
start_epoch = 1
task = 'pansharpening'
tfb_dir = None
use_log = True
use_tfb = False
validate = False
scale = [1]
data_dir = 'D:/Datasets/pansharpening'
best_prec1 = 10000
best_prec5 = 10000
metrics = 'loss'
save_fmt = 'mat'
taskhead = 'pansharpening'
best_epoch = 1
dataset = dict(train='wv3', test='test_wv3_multiExm1.h5')
epochs = 400
eval = False
experimental_desc = 'Test'
lr = 0.0003
out_dir = 'd:/Python/gitSync/AEM/results/pansharpening'
print_freq = 50
resume_from = '.pth.tar'
samples_per_gpu = 32
workers_per_gpu = 

d:\Python\Anaconda3\envs\latest\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
d:\python\gitsync\udl-vis\udl_vis\AutoDL\trainer.py:120: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  'please set `runner` in your config.', UserWarning)
d:\python\gitsync\udl_package\pancollection\pancollection\common\dataset.py:21: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  data = h5py.File(file_path)  # NxCxHxW = 0x1x2x3
d:\python\gitsync\udl-vis\udl_vis\mmcv\runner\checkpoint.py:585: UserWarning: checkpoint in di

(9714, 1, 64, 64) (9714, 8, 64, 64) (9714, 8, 64, 64) (9714, 8, 16, 16)


- Epoch [1]/[400][50/303]	lr: 3.000e-04, eta: 3:32:15, time: 0.105, data_time: 0.046, memory: 203MB, pan2ms: 0.00780, loss: 0.00780, grad_norm: 0.02161
- Epoch [1]/[400][100/303]	lr: 3.000e-04, eta: 3:03:56, time: 0.077, data_time: 0.026, memory: 203MB, pan2ms: 0.00608, loss: 0.00608, grad_norm: 0.01404


KeyboardInterrupt: 

Example2 

In [5]:
from pancollection.run_test_pansharpening import run_test
arch='PNN'
# dataset_name='gf2'
# cfg = dict(arch=arch, dataset_name=dataset_name, eval=True,
            #  dataset={'test': f'test_{dataset_name}_multiExm1_hp.h5'} if arch == "PanNet" else {'test': f'test_{dataset_name}_multiExm1.h5'}))
cfg = dict(arch=arch, eval=True)
run_test(**cfg)

d:\python\gitsync\udl_package\pancollection\pancollection\configs\configs.py:33: UserWarning: Note: FusionNet, DiCNN, PNN don't have high-pass filter
  warnings.warn(warning)
- Set random seed to 1


111 entrypoint
111 None
222 pansharpening
111 None
333 PNN
accumulated_step = 1
amp = None
amp_opt_level = 'O0'
arch = 'PNN'
backend = 'nccl'
clip_max_norm = 0
crop_batch_size = 128
device = 'cuda'
dist_url = 'env://'
global_rank = 0
gpu_ids = [0]
launcher = 'none'
load_model_strict = True
local_rank = 0
log_dir = 'logs'
mode = None
model_style = None
once_epoch = False
reg = True
reset_lr = False
resume_mode = 'best'
rgb_range = 255
save_top_k = 5
seed = 1
start_epoch = 1
task = 'pansharpening'
tfb_dir = None
use_log = True
use_tfb = False
validate = False
scale = [1]
data_dir = 'D:/Datasets/pansharpening'
best_prec1 = 10000
best_prec5 = 10000
metrics = 'loss'
save_fmt = 'mat'
taskhead = 'pansharpening'
best_epoch = 1
dataset = dict(test='test_gf2_multiExm1.h5')
epochs = 12000
eval = True
experimental_desc = 'Test'
lr = 0.001
lr_scheduler = True
out_dir = 'd:/Python/gitSync/AEM/results/pansharpening'
print_freq = 500
resume_from = 'D:/Python/gitSync/ckpts/PanCollection/gf2/PNN/11998.p

d:\Python\Anaconda3\envs\latest\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
d:\python\gitsync\udl-vis\udl_vis\AutoDL\trainer.py:120: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  'please set `runner` in your config.', UserWarning)
d:\python\gitsync\udl_package\pancollection\pancollection\common\dataset.py:144: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  data = h5py.File(file_path)  # CxHxW
d:\python\gitsync\udl-vis\udl_vis\mmcv\runner\checkpoint.py:585: UserWarning: checkpoint in directory d:\

lms: torch.Size([20, 4, 256, 256]), ms: torch.Size([20, 4, 64, 64]), pan: torch.Size([20, 1, 256, 256]), gt: torch.Size([20, 256, 256, 4])


- resumed epoch 11998, iter 3707383
- Start running, host: msi@LAPTOP-JTEEC3T0, work_dir: d:\Python\gitSync\AEM\results\pansharpening\test_gf2_multiExm1.h5\PNN\Test\PNN
- Hooks will be executed in the following order:
before_run:
(NORMAL      ) ModelCheckpoint                    
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) ModelCheckpoint                    
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook   

test time: 10.62796401977539


- Training time 0:00:13


### Step4. Training codes in GaoFen-2 dataset

In [ ]:
%reload_ext autoreload
# %autoreload 2
%aimport udl_vis
%aimport pancollection


import pancollection as pan
cfg = pan.TaskDispatcher.new(task='pansharpening', mode='entrypoint', arch='FusionNet', 
                             dataset_name="gf2", use_resume=False,
                             dataset={'train': 'gf2', 'test': 'test_gf2_multiExm1.h5'})
print(pan.TaskDispatcher._task)
pan.trainer.main(cfg, pan.build_model, pan.getDataSession)


In [2]:
from pancollection.run_pansharpening import run_demo
arch='PNN'
# dataset_name='gf2'
# cfg = dict(arch=arch, dataset_name=dataset_name, use_resume=False,
#              dataset={'train': 'gf2', 'test': 'test_gf2_multiExm1.h5'})
cfg = dict(arch=arch)
run_demo(**cfg)

d:\python\gitsync\udl_package\pancollection\pancollection\configs\configs.py:33: UserWarning: Note: FusionNet, DiCNN, PNN don't have high-pass filter
  warnings.warn(warning)
- Set random seed to 1


111 entrypoint
111 None
222 pansharpening
111 None
333 PNN
accumulated_step = 1
amp = None
amp_opt_level = 'O0'
arch = 'PNN'
backend = 'nccl'
clip_max_norm = 0
crop_batch_size = 128
device = 'cuda'
dist_url = 'env://'
global_rank = 0
gpu_ids = [0]
launcher = 'none'
load_model_strict = True
local_rank = 0
log_dir = 'logs'
mode = None
model_style = None
once_epoch = False
reg = True
reset_lr = False
resume_mode = 'best'
rgb_range = 255
save_top_k = 5
seed = 1
start_epoch = 1
task = 'pansharpening'
tfb_dir = None
use_log = True
use_tfb = False
validate = False
scale = [1]
data_dir = 'D:/Datasets/pansharpening'
best_prec1 = 10000
best_prec5 = 10000
metrics = 'loss'
save_fmt = 'mat'
taskhead = 'pansharpening'
best_epoch = 1
dataset = dict(train='wv3', test='wv3_multiExm1.h5')
epochs = 12000
eval = False
experimental_desc = 'Test'
lr = 0.001
lr_scheduler = True
out_dir = 'd:/Python/gitSync/AEM/results/pansharpening'
print_freq = 500
resume_from = 'D:/Python/gitSync/ckpts/PanCollection/gf2/PN

d:\Python\Anaconda3\envs\latest\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
d:\python\gitsync\udl-vis\udl_vis\AutoDL\trainer.py:120: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  'please set `runner` in your config.', UserWarning)
d:\python\gitsync\udl_package\pancollection\pancollection\common\dataset.py:21: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  data = h5py.File(file_path)  # NxCxHxW = 0x1x2x3
d:\python\gitsync\udl-vis\udl_vis\mmcv\runner\checkpoint.py:585: UserWarning: checkpoint in di

(9714, 1, 64, 64) (9714, 8, 64, 64) (9714, 8, 64, 64) (9714, 8, 16, 16)


RuntimeError: The size of tensor a (5) must match the size of tensor b (9) at non-singleton dimension 1

### `How to custome the new model and the configs in PanCollection`?

In [4]:
from torch import nn
from pancollection.models.FusionNet.model_fusionnet import Resblock


class myFusionNet(nn.Module):
    def __init__(self, spectral_num, criterion, channel=32):
        super(myFusionNet, self).__init__()
        # ConvTranspose2d: output = (input - 1)*stride + outpading - 2*padding + kernelsize
        self.spectral_num = spectral_num
        self.criterion = criterion

        self.conv1 = nn.Conv2d(in_channels=spectral_num, out_channels=channel, kernel_size=3, stride=1, padding=1,
                               bias=True)
        self.res1 = Resblock()
        self.res2 = Resblock()
        self.res3 = Resblock()
        self.res4 = Resblock()

        self.conv3 = nn.Conv2d(in_channels=channel, out_channels=spectral_num, kernel_size=3, stride=1, padding=1,
                               bias=True)

        self.relu = nn.ReLU(inplace=True)

        self.backbone = nn.Sequential(  # method 2: 4 resnet repeated blocks
            self.res1,
            self.res2,
            self.res3,
            self.res4
        )

        # init_weights(self.backbone, self.conv1, self.conv3)   # state initialization, important!
        # self.apply(init_weights)

    def forward(self, x, y):  # x= lms; y = pan

        pan_concat = y.repeat(1, self.spectral_num, 1, 1)  # Bsx8x64x64
        input = torch.sub(pan_concat, x)  # Bsx8x64x64
        rs = self.relu(self.conv1(input))  # Bsx32x64x64

        rs = self.backbone(rs)  # ResNet's backbone!
        output = self.conv3(rs)  # Bsx8x64x64

        return output  # lms + outs

    def train_step(self, data, *args, **kwargs):
        log_vars = {}
        gt, lms, ms, pan = data['gt'].cuda(), data['lms'].cuda(), \
                           data['ms'].cuda(), data['pan'].cuda()
        res = self(lms, pan)
        sr = lms + res  # output:= lms + hp_sr
        loss = self.criterion(sr, gt, *args, **kwargs)['loss']
        # outputs = loss
        # return loss
        log_vars.update(pan2ms=loss.item(), loss=loss.item())
        metrics = {'loss': loss, 'log_vars': log_vars}
        return metrics

    def val_step(self, data, *args, **kwargs):
        # gt, lms, ms, pan = data
        gt, lms, ms, pan = data['gt'].cuda(), data['lms'].cuda(), \
                           data['ms'].cuda(), data['pan'].cuda()
        res = self(lms, pan)
        sr = lms + res  # output:= lms + hp_sr

        return sr, gt

In [5]:
from udl_vis.Basis.criterion_metrics import SetCriterion
from pancollection.models.base_model import PanSharpeningModel

class build_fusionnet(PanSharpeningModel, name='myFusionNet'):
    def __call__(self, args):
        scheduler = None
        if any(["wv" in v for v in args.dataset.values()]):
            spectral_num = 8
        else:
            spectral_num = 4


        loss = nn.MSELoss(size_average=True).cuda()  ## Define the Loss function
        weight_dict = {'loss': 1}
        losses = {'loss': loss}
        criterion = SetCriterion(losses, weight_dict)
        model = myFusionNet(spectral_num, criterion).cuda()
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=0)   ## optimizer 1: Adam

        return model, criterion, optimizer, scheduler

In [23]:
import argparse
from pancollection.configs.configs import TaskDispatcher
import os

class parser_args(TaskDispatcher, name='myFusionNet'):
    def __init__(self, cfg=None, **kwargs):
        super(parser_args, self).__init__()

        if cfg is None:
            from pancollection.configs.configs import panshaprening_cfg
            cfg = panshaprening_cfg()

        
        script_path = os.path.dirname(os.getcwd())
        root_dir = script_path.split(cfg.task)[0]

        model_path = f'.pth.tar'
        dataset_name = kwargs["dataset_name"] if 'dataset_name' in kwargs else "wv3"
        use_resume = kwargs["use_resume"] if 'use_resume' in kwargs else True

        parser = argparse.ArgumentParser(description='PyTorch Pansharpening Training')
        # * Logger
        parser.add_argument('--out_dir', metavar='DIR', default=f'{root_dir}/results/{cfg.task}',
                            help='path to save model')
        parser.add_argument('--mode', default=argparse.SUPPRESS, help='protective declare, please ignore it')

        parser.add_argument('--lr', default=2e-4, type=float)  # 1e-4 2e-4 8
        # parser.add_argument('--lr_scheduler', default=True, type=bool)
        parser.add_argument('--samples_per_gpu', default=32, type=int,  # 8
                            metavar='N', help='mini-batch size (default: 256)')
        parser.add_argument('--print-freq', '-p', default=50, type=int,
                            metavar='N', help='print frequency (default: 10)')
        parser.add_argument('--seed', default=1, type=int,
                            help='seed for initializing training. ')
        parser.add_argument('--epochs', default=400, type=int)
        parser.add_argument('--workers_per_gpu', default=0, type=int)
        parser.add_argument('--resume_from',
                            default=model_path,
                            type=str, metavar='PATH',
                            help='path to latest checkpoint (default: none)')
        ##
        parser.add_argument('--arch', '-a', metavar='ARCH', default='FusionNet', type=str,
                            choices=['PanNet', 'DiCNN', 'PNN', 'FusionNet'])
        parser.add_argument('--dataset', default={'train': 'wv3', 'test': 'test_wv3_multiExm1.h5'}, type=str, # 'valid': 'wv3' , 'eval': 'wv3_multiExm.h5'
                            choices=[None, 'wv2', 'wv3', 'wv4', 'qb', 'gf2', 'wv3_OrigScale_multiExm1.h5', 'test_wv3_multiExm1.h5'],
                            help="performing evalution for patch2entire")
        parser.add_argument('--eval', default=False, type=bool,
                            help="performing evaluation out of training process, which can avoid dead loop !!")


        args = parser.parse_args(args=[])
        args.start_epoch = args.best_epoch = 1
        args.experimental_desc = 'Test'
        cfg.merge_args2cfg(args)
        cfg.save_fmt = "mat"
        # cfg.workflow = [('train', 10), ('val', 1)]
        cfg.workflow = [('train', 1)]
        # cfg.config = f"{script_path}/configs/hook_configs.py"
        cfg.use_tfb = False
        cfg.img_range = 2047.0 if dataset_name != "gf2" else 1023.0
        cfg.dataloader_name = "PanCollection_dataloader"  # PanCollection_dataloader, oldPan_dataloader, DLPan_dataloader
        cfg.merge_args2cfg(args)
        
        cfg.merge_from_dict(kwargs)  # dict is merged partially
        cfg.dataset = kwargs['dataset'] if 'dataset' in kwargs else cfg.dataset
        print(cfg.pretty_text)


        self.merge_from_dict(cfg)


In [24]:
from pancollection import TaskDispatcher, trainer, build_model, getDataSession
dataset_name = "gf2"
arch = "myFusionNet"
cfg = TaskDispatcher.new(task='pansharpening', mode='entrypoint', arch=arch,
                         dataset_name=dataset_name, eval=False, use_resume=False,
                        #  dataset={'test': f'test_{dataset_name}_multiExm1_hp.h5'}
                        #  if arch == "PanNet" else {'test': f'test_{dataset_name}_multiExm1.h5'}
                         ) # 'test_wv3_OrigScale_multiExm1.h5'
print(TaskDispatcher._task.keys())
trainer.main(cfg, build_model, getDataSession)

- Set random seed to 1


111 entrypoint
111 None
222 pansharpening
111 None
333 myFusionNet
accumulated_step = 1
amp = None
amp_opt_level = 'O0'
arch = 'FusionNet'
backend = 'nccl'
clip_max_norm = 0
crop_batch_size = 128
device = 'cuda'
dist_url = 'env://'
global_rank = 0
gpu_ids = [0]
launcher = 'none'
load_model_strict = True
local_rank = 0
log_dir = 'logs'
mode = None
model_style = None
once_epoch = False
reg = True
reset_lr = False
resume_mode = 'best'
rgb_range = 255
save_top_k = 5
seed = 1
start_epoch = 1
task = 'pansharpening'
tfb_dir = None
use_log = True
use_tfb = False
validate = False
scale = [1]
data_dir = 'D:/Datasets/pansharpening'
best_prec1 = 10000
best_prec5 = 10000
metrics = 'loss'
save_fmt = 'mat'
taskhead = 'pansharpening'
best_epoch = 1
dataset = dict(train='wv3', test='test_wv3_multiExm1.h5')
epochs = 400
eval = False
experimental_desc = 'Test'
lr = 0.0002
out_dir = 'd:/Python/gitSync/AEM/results/pansharpening'
print_freq = 50
resume_from = '.pth.tar'
samples_per_gpu = 32
workers_per_gpu 

d:\Python\Anaconda3\envs\latest\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
d:\python\gitsync\udl-vis\udl_vis\AutoDL\trainer.py:120: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  'please set `runner` in your config.', UserWarning)
d:\python\gitsync\udl_package\pancollection\pancollection\common\dataset.py:21: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  data = h5py.File(file_path)  # NxCxHxW = 0x1x2x3
d:\python\gitsync\udl-vis\udl_vis\mmcv\runner\checkpoint.py:585: UserWarning: checkpoint in di

(9714, 1, 64, 64) (9714, 8, 64, 64) (9714, 8, 64, 64) (9714, 8, 16, 16)


- Epoch [1]/[400][50/303]	lr: 2.000e-04, eta: 3:57:48, time: 0.118, data_time: 0.049, memory: 305MB, pan2ms: 0.00879, loss: 0.00879, grad_norm: 0.02547


KeyboardInterrupt: 